In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pywt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.lines import Line2D

file_path = '/home/ab/Desktop/MASTER DSEF/Dogecoin Historical Data_.ods'
db = pd.read_excel(file_path)

# Convertir la colonne 'Date' en format datetime en utilisant le format correct
db['Date'] = pd.to_datetime(db['Date'], format='%m/%d/%Y')

# Inverser l'ordre des données (de la plus ancienne à la plus récente)
data = db.iloc[::-1].reset_index(drop=True)

#data = data[:-1] # sup

date_column = 'Date'
close_column = 'Price'

dates = pd.to_datetime(data[date_column])
close_prices = data[close_column]

# Décomposition en ondelettes
coeffs = pywt.wavedec(close_prices, 'db4', level=2)
cA2, cD2, cD1 = coeffs

approximation = pywt.upcoef('a', cA2, 'db4', level=2, take=len(close_prices))
detail2 = pywt.upcoef('d', cD2, 'db4', level=2, take=len(close_prices))
detail1 = pywt.upcoef('d', cD1, 'db4', level=1, take=len(close_prices))

import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


# Normalisation des données
scaler = MinMaxScaler(feature_range=(0, 1))
cD1_scaled = scaler.fit_transform(cD1.reshape(-1, 1))

# Création des fenêtres temporelles
def create_dataset(data, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i+window_size, 0])  # Séquence des données passées
        y.append(data[i+window_size, 0])  # Prochaine valeur (target)
    return np.array(X), np.array(y)

window_size = 10  # Utilisez les 10 dernières valeurs pour prédire la suivante
X, y = create_dataset(cD1_scaled, window_size)

# Reshape des données pour LSTM [échantillons, timesteps, features]
X = X.reshape(X.shape[0], X.shape[1], 1)

# Division en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

from keras.models import Sequential
from keras.layers import LSTM, Dense

# Définition du modèle LSTM
model = Sequential()
model.add(LSTM(units=50, return_sequences=False, input_shape=(X_train.shape[1], 1)))
model.add(Dense(1))  # Une seule sortie (la prochaine valeur de cD1)

# Compilation du modèle
model.compile(optimizer='adam', loss='mean_squared_error')

# Entraînement du modèle
model.fit(X_train, y_train, epochs=20, batch_size=32)

# Prédire la prochaine valeur de cD1 à partir de la dernière fenêtre de test
predicted_value_cD1 = model.predict(X_test[-1].reshape(1, window_size, 1))

# Inverse de la normalisation pour obtenir la valeur réelle
predicted_value_rescaled_cD1 = scaler.inverse_transform(predicted_value_cD1)

# Affichage de la valeur prédite
print(f"Prochaine valeur prédite pour cD1 : {predicted_value_rescaled_cD1[0][0]}")

# Dernière valeur de cA1 (approximation à un certain niveau)
last_cD1_value = cD1[-1]  # Dernière valeur de cA1

# Valeur prédite rééchelonnée (prédiction pour le prochain point)
predicted_value_rescaled_cD1 = predicted_value_rescaled_cD1[0][0]

# Comparaison entre la dernière valeur de cA1 et la valeur prédite
if predicted_value_rescaled_cD1 > last_cD1_value:
    print("Le signal devrait augmenter.")
else:
    print("Le signal devrait diminuer.")

# Normalisation des données pour cD2
scaler = MinMaxScaler(feature_range=(0, 1))
cD2_scaled = scaler.fit_transform(cD2.reshape(-1, 1))  # Remplacez cD2 par vos données de cD2

# Création des fenêtres temporelles pour cD2
def create_dataset(data, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i+window_size, 0])  # Séquence des données passées
        y.append(data[i+window_size, 0])  # Prochaine valeur (target)
    return np.array(X), np.array(y)

window_size = 10  # Utilisez les 10 dernières valeurs pour prédire la suivante
X, y = create_dataset(cD2_scaled, window_size)

# Reshape des données pour LSTM [échantillons, timesteps, features]
X = X.reshape(X.shape[0], X.shape[1], 1)

# Division en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Définition du modèle LSTM
model = Sequential()
model.add(LSTM(units=50, return_sequences=False, input_shape=(X_train.shape[1], 1)))
model.add(Dense(1))  # Une seule sortie (la prochaine valeur de cD2)

# Compilation du modèle
model.compile(optimizer='adam', loss='mean_squared_error')

# Entraînement du modèle
model.fit(X_train, y_train, epochs=20, batch_size=32)

# Prédire la prochaine valeur de cD2 à partir de la dernière fenêtre de test
predicted_value_cD2 = model.predict(X_test[-1].reshape(1, window_size, 1))

# Inverse de la normalisation pour obtenir la valeur réelle
predicted_value_rescaled_cD2 = scaler.inverse_transform(predicted_value_cD2)

# Affichage de la valeur prédite
print(f"Prochaine valeur prédite pour cD2 : {predicted_value_rescaled_cD2[0][0]}")

# Dernière valeur de cA1 (approximation à un certain niveau)
last_cD2_value = cD2[-1]  # Dernière valeur de cD2

# Valeur prédite rééchelonnée (prédiction pour le prochain point)
predicted_value_rescaled_cD2 = predicted_value_rescaled_cD2[0][0]

# Comparaison entre la dernière valeur de cA1 et la valeur prédite
if predicted_value_rescaled_cD2 > last_cD2_value:
    print("Le signal devrait augmenter.")
else:
    print("Le signal devrait diminuer.")

# Définir les valeurs à comparer
last_cD2_value = cD2[-1]  # Dernière valeur de cD2
last_cD1_value = cD1[-1]  # Dernière valeur de cD1
predicted_value_rescaled_cD1 = predicted_value_rescaled_cD1  # Prédiction pour cD1
predicted_value_rescaled_cD2 = predicted_value_rescaled_cD2  # Prédiction pour cD2

# Condition pour Achat, Vente ou Neutre
if last_cD2_value < predicted_value_rescaled_cD2 and last_cD1_value < predicted_value_rescaled_cD1:
    print("Achetez")
elif last_cD2_value >= predicted_value_rescaled_cD2 and last_cD1_value >= predicted_value_rescaled_cD2:
    print("N'achetez pas") # vente
else:
    print("N'achetez pas") # neutre

Epoch 1/20


/home/ab/Desktop/qiskit-env/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.1546
Epoch 2/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0028   
Epoch 3/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0023   
Epoch 4/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0016   
Epoch 5/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0016  
Epoch 6/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0012   
Epoch 7/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0013  
Epoch 8/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0016  
Epoch 9/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0012   
Epoch 10/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0018
Epoch 11/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0021  
Epoch 12/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0014   
Epoch 13/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0012   
Epoch 14/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0013   
Epoch 15/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.001

/home/ab/Desktop/qiskit-env/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.1733
Epoch 2/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0116
Epoch 3/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0042
Epoch 4/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0039
Epoch 5/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0034
Epoch 6/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0021
Epoch 7/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0027    
Epoch 8/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0030
Epoch 9/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0040
Epoch 10/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0028
Epoch 11/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0022    
Epoch 12/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0024    
Epoch 13/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0026    
Epoch 14/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0020
Epoch 15/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0042
Epoch 16/20
18

In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pywt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.lines import Line2D

file_path = '/home/ab/Desktop/MASTER DSEF/Dogecoin Historical Data_.ods'
db = pd.read_excel(file_path)

# Convertir la colonne 'Date' en format datetime en utilisant le format correct
db['Date'] = pd.to_datetime(db['Date'], format='%m/%d/%Y')

# Inverser l'ordre des données (de la plus ancienne à la plus récente)
data = db.iloc[::-1].reset_index(drop=True)

data = data[:-1] # sup

date_column = 'Date'
close_column = 'Price'

dates = pd.to_datetime(data[date_column])
close_prices = data[close_column]

# Décomposition en ondelettes
coeffs = pywt.wavedec(close_prices, 'db4', level=2)
cA2, cD2, cD1 = coeffs

approximation = pywt.upcoef('a', cA2, 'db4', level=2, take=len(close_prices))
detail2 = pywt.upcoef('d', cD2, 'db4', level=2, take=len(close_prices))
detail1 = pywt.upcoef('d', cD1, 'db4', level=1, take=len(close_prices))

import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


# Normalisation des données
scaler = MinMaxScaler(feature_range=(0, 1))
cD1_scaled = scaler.fit_transform(cD1.reshape(-1, 1))

# Création des fenêtres temporelles
def create_dataset(data, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i+window_size, 0])  # Séquence des données passées
        y.append(data[i+window_size, 0])  # Prochaine valeur (target)
    return np.array(X), np.array(y)

window_size = 10  # Utilisez les 10 dernières valeurs pour prédire la suivante
X, y = create_dataset(cD1_scaled, window_size)

# Reshape des données pour LSTM [échantillons, timesteps, features]
X = X.reshape(X.shape[0], X.shape[1], 1)

# Division en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

from keras.models import Sequential
from keras.layers import LSTM, Dense

# Définition du modèle LSTM
model = Sequential()
model.add(LSTM(units=50, return_sequences=False, input_shape=(X_train.shape[1], 1)))
model.add(Dense(1))  # Une seule sortie (la prochaine valeur de cD1)

# Compilation du modèle
model.compile(optimizer='adam', loss='mean_squared_error')

# Entraînement du modèle
model.fit(X_train, y_train, epochs=20, batch_size=32)

# Prédire la prochaine valeur de cD1 à partir de la dernière fenêtre de test
predicted_value_cD1 = model.predict(X_test[-1].reshape(1, window_size, 1))

# Inverse de la normalisation pour obtenir la valeur réelle
predicted_value_rescaled_cD1 = scaler.inverse_transform(predicted_value_cD1)

# Affichage de la valeur prédite
print(f"Prochaine valeur prédite pour cD1 : {predicted_value_rescaled_cD1[0][0]}")

# Dernière valeur de cA1 (approximation à un certain niveau)
last_cD1_value = cD1[-1]  # Dernière valeur de cA1

# Valeur prédite rééchelonnée (prédiction pour le prochain point)
predicted_value_rescaled_cD1 = predicted_value_rescaled_cD1[0][0]

# Comparaison entre la dernière valeur de cA1 et la valeur prédite
if predicted_value_rescaled_cD1 > last_cD1_value:
    print("Le signal devrait augmenter.")
else:
    print("Le signal devrait diminuer.")

# Normalisation des données pour cD2
scaler = MinMaxScaler(feature_range=(0, 1))
cD2_scaled = scaler.fit_transform(cD2.reshape(-1, 1))  # Remplacez cD2 par vos données de cD2

# Création des fenêtres temporelles pour cD2
def create_dataset(data, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i+window_size, 0])  # Séquence des données passées
        y.append(data[i+window_size, 0])  # Prochaine valeur (target)
    return np.array(X), np.array(y)

window_size = 10  # Utilisez les 10 dernières valeurs pour prédire la suivante
X, y = create_dataset(cD2_scaled, window_size)

# Reshape des données pour LSTM [échantillons, timesteps, features]
X = X.reshape(X.shape[0], X.shape[1], 1)

# Division en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Définition du modèle LSTM
model = Sequential()
model.add(LSTM(units=50, return_sequences=False, input_shape=(X_train.shape[1], 1)))
model.add(Dense(1))  # Une seule sortie (la prochaine valeur de cD2)

# Compilation du modèle
model.compile(optimizer='adam', loss='mean_squared_error')

# Entraînement du modèle
model.fit(X_train, y_train, epochs=20, batch_size=32)

# Prédire la prochaine valeur de cD2 à partir de la dernière fenêtre de test
predicted_value_cD2 = model.predict(X_test[-1].reshape(1, window_size, 1))

# Inverse de la normalisation pour obtenir la valeur réelle
predicted_value_rescaled_cD2 = scaler.inverse_transform(predicted_value_cD2)

# Affichage de la valeur prédite
print(f"Prochaine valeur prédite pour cD2 : {predicted_value_rescaled_cD2[0][0]}")

# Dernière valeur de cA1 (approximation à un certain niveau)
last_cD2_value = cD2[-1]  # Dernière valeur de cD2

# Valeur prédite rééchelonnée (prédiction pour le prochain point)
predicted_value_rescaled_cD2 = predicted_value_rescaled_cD2[0][0]

# Comparaison entre la dernière valeur de cA1 et la valeur prédite
if predicted_value_rescaled_cD2 > last_cD2_value:
    print("Le signal devrait augmenter.")
else:
    print("Le signal devrait diminuer.")

# Définir les valeurs à comparer
last_cD2_value = cD2[-1]  # Dernière valeur de cD2
last_cD1_value = cD1[-1]  # Dernière valeur de cD1
predicted_value_rescaled_cD1 = predicted_value_rescaled_cD1  # Prédiction pour cD1
predicted_value_rescaled_cD2 = predicted_value_rescaled_cD2  # Prédiction pour cD2

# Condition pour Achat, Vente ou Neutre
if last_cD2_value < predicted_value_rescaled_cD2 and last_cD1_value < predicted_value_rescaled_cD1:
    print("Achetez")
elif last_cD2_value >= predicted_value_rescaled_cD2 and last_cD1_value >= predicted_value_rescaled_cD2:
    print("N'achetez pas") # vente
else:
    print("N'achetez pas") # neutre

Epoch 1/20


/home/ab/Desktop/qiskit-env/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.1011
Epoch 2/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0033
Epoch 3/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0014  
Epoch 4/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0011    
Epoch 5/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0014  
Epoch 6/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0019   
Epoch 7/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0016   
Epoch 8/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0017   
Epoch 9/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0018
Epoch 10/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0015
Epoch 11/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0014   
Epoch 12/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0021   
Epoch 13/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0017   
Epoch 14/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0026
Epoch 15/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0021  
Ep

/home/ab/Desktop/qiskit-env/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0878
Epoch 2/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0082
Epoch 3/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0024
Epoch 4/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0032
Epoch 5/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0025
Epoch 6/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0041
Epoch 7/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0021
Epoch 8/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0035
Epoch 9/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0018    
Epoch 10/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0037
Epoch 11/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0023
Epoch 12/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0025
Epoch 13/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0028
Epoch 14/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0036
Epoch 15/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0018    
Epoch 16/20
18/18 ━━━━

In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pywt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.lines import Line2D

file_path = '/home/ab/Desktop/MASTER DSEF/Dogecoin Historical Data_.ods'
db = pd.read_excel(file_path)

# Convertir la colonne 'Date' en format datetime en utilisant le format correct
db['Date'] = pd.to_datetime(db['Date'], format='%m/%d/%Y')

# Inverser l'ordre des données (de la plus ancienne à la plus récente)
data = db.iloc[::-1].reset_index(drop=True)

data = data[:-2] # sup

date_column = 'Date'
close_column = 'Price'

dates = pd.to_datetime(data[date_column])
close_prices = data[close_column]

# Décomposition en ondelettes
coeffs = pywt.wavedec(close_prices, 'db4', level=2)
cA2, cD2, cD1 = coeffs

approximation = pywt.upcoef('a', cA2, 'db4', level=2, take=len(close_prices))
detail2 = pywt.upcoef('d', cD2, 'db4', level=2, take=len(close_prices))
detail1 = pywt.upcoef('d', cD1, 'db4', level=1, take=len(close_prices))

import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


# Normalisation des données
scaler = MinMaxScaler(feature_range=(0, 1))
cD1_scaled = scaler.fit_transform(cD1.reshape(-1, 1))

# Création des fenêtres temporelles
def create_dataset(data, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i+window_size, 0])  # Séquence des données passées
        y.append(data[i+window_size, 0])  # Prochaine valeur (target)
    return np.array(X), np.array(y)

window_size = 10  # Utilisez les 10 dernières valeurs pour prédire la suivante
X, y = create_dataset(cD1_scaled, window_size)

# Reshape des données pour LSTM [échantillons, timesteps, features]
X = X.reshape(X.shape[0], X.shape[1], 1)

# Division en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

from keras.models import Sequential
from keras.layers import LSTM, Dense

# Définition du modèle LSTM
model = Sequential()
model.add(LSTM(units=50, return_sequences=False, input_shape=(X_train.shape[1], 1)))
model.add(Dense(1))  # Une seule sortie (la prochaine valeur de cD1)

# Compilation du modèle
model.compile(optimizer='adam', loss='mean_squared_error')

# Entraînement du modèle
model.fit(X_train, y_train, epochs=20, batch_size=32)

# Prédire la prochaine valeur de cD1 à partir de la dernière fenêtre de test
predicted_value_cD1 = model.predict(X_test[-1].reshape(1, window_size, 1))

# Inverse de la normalisation pour obtenir la valeur réelle
predicted_value_rescaled_cD1 = scaler.inverse_transform(predicted_value_cD1)

# Affichage de la valeur prédite
print(f"Prochaine valeur prédite pour cD1 : {predicted_value_rescaled_cD1[0][0]}")

# Dernière valeur de cA1 (approximation à un certain niveau)
last_cD1_value = cD1[-1]  # Dernière valeur de cA1

# Valeur prédite rééchelonnée (prédiction pour le prochain point)
predicted_value_rescaled_cD1 = predicted_value_rescaled_cD1[0][0]

# Comparaison entre la dernière valeur de cA1 et la valeur prédite
if predicted_value_rescaled_cD1 > last_cD1_value:
    print("Le signal devrait augmenter.")
else:
    print("Le signal devrait diminuer.")

# Normalisation des données pour cD2
scaler = MinMaxScaler(feature_range=(0, 1))
cD2_scaled = scaler.fit_transform(cD2.reshape(-1, 1))  # Remplacez cD2 par vos données de cD2

# Création des fenêtres temporelles pour cD2
def create_dataset(data, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i+window_size, 0])  # Séquence des données passées
        y.append(data[i+window_size, 0])  # Prochaine valeur (target)
    return np.array(X), np.array(y)

window_size = 10  # Utilisez les 10 dernières valeurs pour prédire la suivante
X, y = create_dataset(cD2_scaled, window_size)

# Reshape des données pour LSTM [échantillons, timesteps, features]
X = X.reshape(X.shape[0], X.shape[1], 1)

# Division en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Définition du modèle LSTM
model = Sequential()
model.add(LSTM(units=50, return_sequences=False, input_shape=(X_train.shape[1], 1)))
model.add(Dense(1))  # Une seule sortie (la prochaine valeur de cD2)

# Compilation du modèle
model.compile(optimizer='adam', loss='mean_squared_error')

# Entraînement du modèle
model.fit(X_train, y_train, epochs=20, batch_size=32)

# Prédire la prochaine valeur de cD2 à partir de la dernière fenêtre de test
predicted_value_cD2 = model.predict(X_test[-1].reshape(1, window_size, 1))

# Inverse de la normalisation pour obtenir la valeur réelle
predicted_value_rescaled_cD2 = scaler.inverse_transform(predicted_value_cD2)

# Affichage de la valeur prédite
print(f"Prochaine valeur prédite pour cD2 : {predicted_value_rescaled_cD2[0][0]}")

# Dernière valeur de cA1 (approximation à un certain niveau)
last_cD2_value = cD2[-1]  # Dernière valeur de cD2

# Valeur prédite rééchelonnée (prédiction pour le prochain point)
predicted_value_rescaled_cD2 = predicted_value_rescaled_cD2[0][0]

# Comparaison entre la dernière valeur de cA1 et la valeur prédite
if predicted_value_rescaled_cD2 > last_cD2_value:
    print("Le signal devrait augmenter.")
else:
    print("Le signal devrait diminuer.")

# Définir les valeurs à comparer
last_cD2_value = cD2[-1]  # Dernière valeur de cD2
last_cD1_value = cD1[-1]  # Dernière valeur de cD1
predicted_value_rescaled_cD1 = predicted_value_rescaled_cD1  # Prédiction pour cD1
predicted_value_rescaled_cD2 = predicted_value_rescaled_cD2  # Prédiction pour cD2

# Condition pour Achat, Vente ou Neutre
if last_cD2_value < predicted_value_rescaled_cD2 and last_cD1_value < predicted_value_rescaled_cD1:
    print("Achetez")
elif last_cD2_value >= predicted_value_rescaled_cD2 and last_cD1_value >= predicted_value_rescaled_cD2:
    print("N'achetez pas") # vente
else:
    print("N'achetez pas") # neutre

Epoch 1/20


/home/ab/Desktop/qiskit-env/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.0826
Epoch 2/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0024
Epoch 3/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0012   
Epoch 4/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0017  
Epoch 5/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0011    
Epoch 6/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0020
Epoch 7/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0015 
Epoch 8/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0018
Epoch 9/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0016  
Epoch 10/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0011    
Epoch 11/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0012   
Epoch 12/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0012   
Epoch 13/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 9.6779e-04
Epoch 14/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0025  
Epoch 15/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0021
E

/home/ab/Desktop/qiskit-env/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.0794
Epoch 2/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0071
Epoch 3/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0037
Epoch 4/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0046
Epoch 5/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0039
Epoch 6/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0024    
Epoch 7/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0031  
Epoch 8/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0041
Epoch 9/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0023
Epoch 10/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0034
Epoch 11/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0039
Epoch 12/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0019
Epoch 13/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0025
Epoch 14/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0035
Epoch 15/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0042
Epoch 16/20
18/18

In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pywt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.lines import Line2D

file_path = '/home/ab/Desktop/MASTER DSEF/Dogecoin Historical Data_.ods'
db = pd.read_excel(file_path)

# Convertir la colonne 'Date' en format datetime en utilisant le format correct
db['Date'] = pd.to_datetime(db['Date'], format='%m/%d/%Y')

# Inverser l'ordre des données (de la plus ancienne à la plus récente)
data = db.iloc[::-1].reset_index(drop=True)

data = data[:-3] # sup

date_column = 'Date'
close_column = 'Price'

dates = pd.to_datetime(data[date_column])
close_prices = data[close_column]

# Décomposition en ondelettes
coeffs = pywt.wavedec(close_prices, 'db4', level=2)
cA2, cD2, cD1 = coeffs

approximation = pywt.upcoef('a', cA2, 'db4', level=2, take=len(close_prices))
detail2 = pywt.upcoef('d', cD2, 'db4', level=2, take=len(close_prices))
detail1 = pywt.upcoef('d', cD1, 'db4', level=1, take=len(close_prices))

import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


# Normalisation des données
scaler = MinMaxScaler(feature_range=(0, 1))
cD1_scaled = scaler.fit_transform(cD1.reshape(-1, 1))

# Création des fenêtres temporelles
def create_dataset(data, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i+window_size, 0])  # Séquence des données passées
        y.append(data[i+window_size, 0])  # Prochaine valeur (target)
    return np.array(X), np.array(y)

window_size = 10  # Utilisez les 10 dernières valeurs pour prédire la suivante
X, y = create_dataset(cD1_scaled, window_size)

# Reshape des données pour LSTM [échantillons, timesteps, features]
X = X.reshape(X.shape[0], X.shape[1], 1)

# Division en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

from keras.models import Sequential
from keras.layers import LSTM, Dense

# Définition du modèle LSTM
model = Sequential()
model.add(LSTM(units=50, return_sequences=False, input_shape=(X_train.shape[1], 1)))
model.add(Dense(1))  # Une seule sortie (la prochaine valeur de cD1)

# Compilation du modèle
model.compile(optimizer='adam', loss='mean_squared_error')

# Entraînement du modèle
model.fit(X_train, y_train, epochs=20, batch_size=32)

# Prédire la prochaine valeur de cD1 à partir de la dernière fenêtre de test
predicted_value_cD1 = model.predict(X_test[-1].reshape(1, window_size, 1))

# Inverse de la normalisation pour obtenir la valeur réelle
predicted_value_rescaled_cD1 = scaler.inverse_transform(predicted_value_cD1)

# Affichage de la valeur prédite
print(f"Prochaine valeur prédite pour cD1 : {predicted_value_rescaled_cD1[0][0]}")

# Dernière valeur de cA1 (approximation à un certain niveau)
last_cD1_value = cD1[-1]  # Dernière valeur de cA1

# Valeur prédite rééchelonnée (prédiction pour le prochain point)
predicted_value_rescaled_cD1 = predicted_value_rescaled_cD1[0][0]

# Comparaison entre la dernière valeur de cA1 et la valeur prédite
if predicted_value_rescaled_cD1 > last_cD1_value:
    print("Le signal devrait augmenter.")
else:
    print("Le signal devrait diminuer.")

# Normalisation des données pour cD2
scaler = MinMaxScaler(feature_range=(0, 1))
cD2_scaled = scaler.fit_transform(cD2.reshape(-1, 1))  # Remplacez cD2 par vos données de cD2

# Création des fenêtres temporelles pour cD2
def create_dataset(data, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i+window_size, 0])  # Séquence des données passées
        y.append(data[i+window_size, 0])  # Prochaine valeur (target)
    return np.array(X), np.array(y)

window_size = 10  # Utilisez les 10 dernières valeurs pour prédire la suivante
X, y = create_dataset(cD2_scaled, window_size)

# Reshape des données pour LSTM [échantillons, timesteps, features]
X = X.reshape(X.shape[0], X.shape[1], 1)

# Division en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Définition du modèle LSTM
model = Sequential()
model.add(LSTM(units=50, return_sequences=False, input_shape=(X_train.shape[1], 1)))
model.add(Dense(1))  # Une seule sortie (la prochaine valeur de cD2)

# Compilation du modèle
model.compile(optimizer='adam', loss='mean_squared_error')

# Entraînement du modèle
model.fit(X_train, y_train, epochs=20, batch_size=32)

# Prédire la prochaine valeur de cD2 à partir de la dernière fenêtre de test
predicted_value_cD2 = model.predict(X_test[-1].reshape(1, window_size, 1))

# Inverse de la normalisation pour obtenir la valeur réelle
predicted_value_rescaled_cD2 = scaler.inverse_transform(predicted_value_cD2)

# Affichage de la valeur prédite
print(f"Prochaine valeur prédite pour cD2 : {predicted_value_rescaled_cD2[0][0]}")

# Dernière valeur de cA1 (approximation à un certain niveau)
last_cD2_value = cD2[-1]  # Dernière valeur de cD2

# Valeur prédite rééchelonnée (prédiction pour le prochain point)
predicted_value_rescaled_cD2 = predicted_value_rescaled_cD2[0][0]

# Comparaison entre la dernière valeur de cA1 et la valeur prédite
if predicted_value_rescaled_cD2 > last_cD2_value:
    print("Le signal devrait augmenter.")
else:
    print("Le signal devrait diminuer.")

# Définir les valeurs à comparer
last_cD2_value = cD2[-1]  # Dernière valeur de cD2
last_cD1_value = cD1[-1]  # Dernière valeur de cD1
predicted_value_rescaled_cD1 = predicted_value_rescaled_cD1  # Prédiction pour cD1
predicted_value_rescaled_cD2 = predicted_value_rescaled_cD2  # Prédiction pour cD2

# Condition pour Achat, Vente ou Neutre
if last_cD2_value < predicted_value_rescaled_cD2 and last_cD1_value < predicted_value_rescaled_cD1:
    print("Achetez")
elif last_cD2_value >= predicted_value_rescaled_cD2 and last_cD1_value >= predicted_value_rescaled_cD2:
    print("N'achetez pas") # vente
else:
    print("N'achetez pas") # neutre

Epoch 1/20


/home/ab/Desktop/qiskit-env/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.1100
Epoch 2/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0026
Epoch 3/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0013  
Epoch 4/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0021
Epoch 5/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0013   
Epoch 6/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0017
Epoch 7/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0013   
Epoch 8/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0025
Epoch 9/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0015  
Epoch 10/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 9.8726e-04
Epoch 11/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0013 
Epoch 12/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0014  
Epoch 13/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0019
Epoch 14/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0022
Epoch 15/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0017   
Epoch 16/2

/home/ab/Desktop/qiskit-env/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.1209
Epoch 2/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0102
Epoch 3/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0036
Epoch 4/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0040
Epoch 5/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0022
Epoch 6/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0023   
Epoch 7/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0050
Epoch 8/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0026
Epoch 9/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0032
Epoch 10/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0024
Epoch 11/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0027
Epoch 12/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0031
Epoch 13/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0018   
Epoch 14/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0025
Epoch 15/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0013   
Epoch 16/20
18/1

In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pywt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.lines import Line2D

file_path = '/home/ab/Desktop/MASTER DSEF/Dogecoin Historical Data_.ods'
db = pd.read_excel(file_path)

# Convertir la colonne 'Date' en format datetime en utilisant le format correct
db['Date'] = pd.to_datetime(db['Date'], format='%m/%d/%Y')

# Inverser l'ordre des données (de la plus ancienne à la plus récente)
data = db.iloc[::-1].reset_index(drop=True)

data = data[:-4] # sup

date_column = 'Date'
close_column = 'Price'

dates = pd.to_datetime(data[date_column])
close_prices = data[close_column]

# Décomposition en ondelettes
coeffs = pywt.wavedec(close_prices, 'db4', level=2)
cA2, cD2, cD1 = coeffs

approximation = pywt.upcoef('a', cA2, 'db4', level=2, take=len(close_prices))
detail2 = pywt.upcoef('d', cD2, 'db4', level=2, take=len(close_prices))
detail1 = pywt.upcoef('d', cD1, 'db4', level=1, take=len(close_prices))

import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


# Normalisation des données
scaler = MinMaxScaler(feature_range=(0, 1))
cD1_scaled = scaler.fit_transform(cD1.reshape(-1, 1))

# Création des fenêtres temporelles
def create_dataset(data, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i+window_size, 0])  # Séquence des données passées
        y.append(data[i+window_size, 0])  # Prochaine valeur (target)
    return np.array(X), np.array(y)

window_size = 10  # Utilisez les 10 dernières valeurs pour prédire la suivante
X, y = create_dataset(cD1_scaled, window_size)

# Reshape des données pour LSTM [échantillons, timesteps, features]
X = X.reshape(X.shape[0], X.shape[1], 1)

# Division en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

from keras.models import Sequential
from keras.layers import LSTM, Dense

# Définition du modèle LSTM
model = Sequential()
model.add(LSTM(units=50, return_sequences=False, input_shape=(X_train.shape[1], 1)))
model.add(Dense(1))  # Une seule sortie (la prochaine valeur de cD1)

# Compilation du modèle
model.compile(optimizer='adam', loss='mean_squared_error')

# Entraînement du modèle
model.fit(X_train, y_train, epochs=20, batch_size=32)

# Prédire la prochaine valeur de cD1 à partir de la dernière fenêtre de test
predicted_value_cD1 = model.predict(X_test[-1].reshape(1, window_size, 1))

# Inverse de la normalisation pour obtenir la valeur réelle
predicted_value_rescaled_cD1 = scaler.inverse_transform(predicted_value_cD1)

# Affichage de la valeur prédite
print(f"Prochaine valeur prédite pour cD1 : {predicted_value_rescaled_cD1[0][0]}")

# Dernière valeur de cA1 (approximation à un certain niveau)
last_cD1_value = cD1[-1]  # Dernière valeur de cA1

# Valeur prédite rééchelonnée (prédiction pour le prochain point)
predicted_value_rescaled_cD1 = predicted_value_rescaled_cD1[0][0]

# Comparaison entre la dernière valeur de cA1 et la valeur prédite
if predicted_value_rescaled_cD1 > last_cD1_value:
    print("Le signal devrait augmenter.")
else:
    print("Le signal devrait diminuer.")

# Normalisation des données pour cD2
scaler = MinMaxScaler(feature_range=(0, 1))
cD2_scaled = scaler.fit_transform(cD2.reshape(-1, 1))  # Remplacez cD2 par vos données de cD2

# Création des fenêtres temporelles pour cD2
def create_dataset(data, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i+window_size, 0])  # Séquence des données passées
        y.append(data[i+window_size, 0])  # Prochaine valeur (target)
    return np.array(X), np.array(y)

window_size = 10  # Utilisez les 10 dernières valeurs pour prédire la suivante
X, y = create_dataset(cD2_scaled, window_size)

# Reshape des données pour LSTM [échantillons, timesteps, features]
X = X.reshape(X.shape[0], X.shape[1], 1)

# Division en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Définition du modèle LSTM
model = Sequential()
model.add(LSTM(units=50, return_sequences=False, input_shape=(X_train.shape[1], 1)))
model.add(Dense(1))  # Une seule sortie (la prochaine valeur de cD2)

# Compilation du modèle
model.compile(optimizer='adam', loss='mean_squared_error')

# Entraînement du modèle
model.fit(X_train, y_train, epochs=20, batch_size=32)

# Prédire la prochaine valeur de cD2 à partir de la dernière fenêtre de test
predicted_value_cD2 = model.predict(X_test[-1].reshape(1, window_size, 1))

# Inverse de la normalisation pour obtenir la valeur réelle
predicted_value_rescaled_cD2 = scaler.inverse_transform(predicted_value_cD2)

# Affichage de la valeur prédite
print(f"Prochaine valeur prédite pour cD2 : {predicted_value_rescaled_cD2[0][0]}")

# Dernière valeur de cA1 (approximation à un certain niveau)
last_cD2_value = cD2[-1]  # Dernière valeur de cD2

# Valeur prédite rééchelonnée (prédiction pour le prochain point)
predicted_value_rescaled_cD2 = predicted_value_rescaled_cD2[0][0]

# Comparaison entre la dernière valeur de cA1 et la valeur prédite
if predicted_value_rescaled_cD2 > last_cD2_value:
    print("Le signal devrait augmenter.")
else:
    print("Le signal devrait diminuer.")

# Définir les valeurs à comparer
last_cD2_value = cD2[-1]  # Dernière valeur de cD2
last_cD1_value = cD1[-1]  # Dernière valeur de cD1
predicted_value_rescaled_cD1 = predicted_value_rescaled_cD1  # Prédiction pour cD1
predicted_value_rescaled_cD2 = predicted_value_rescaled_cD2  # Prédiction pour cD2

# Condition pour Achat, Vente ou Neutre
if last_cD2_value < predicted_value_rescaled_cD2 and last_cD1_value < predicted_value_rescaled_cD1:
    print("Achetez")
elif last_cD2_value >= predicted_value_rescaled_cD2 and last_cD1_value >= predicted_value_rescaled_cD2:
    print("N'achetez pas") # vente
else:
    print("N'achetez pas") # neutre

Epoch 1/20


/home/ab/Desktop/qiskit-env/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0985
Epoch 2/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0040
Epoch 3/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0012  
Epoch 4/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0014
Epoch 5/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0016  
Epoch 6/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0016 
Epoch 7/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0014 
Epoch 8/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0012  
Epoch 9/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0016 
Epoch 10/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0019  
Epoch 11/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0016 
Epoch 12/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0022  
Epoch 13/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0017
Epoch 14/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0019
Epoch 15/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0019 
Epoch 16/20
3

/home/ab/Desktop/qiskit-env/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.1136
Epoch 2/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0129
Epoch 3/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0041
Epoch 4/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0029
Epoch 5/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0027
Epoch 6/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0032
Epoch 7/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0029
Epoch 8/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0034
Epoch 9/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0022    
Epoch 10/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0022
Epoch 11/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0024
Epoch 12/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0020   
Epoch 13/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0031    
Epoch 14/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0026
Epoch 15/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0023
Epoch 16/20
18/18 ━

In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pywt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.lines import Line2D

file_path = '/home/ab/Desktop/MASTER DSEF/Dogecoin Historical Data_.ods'
db = pd.read_excel(file_path)

# Convertir la colonne 'Date' en format datetime en utilisant le format correct
db['Date'] = pd.to_datetime(db['Date'], format='%m/%d/%Y')

# Inverser l'ordre des données (de la plus ancienne à la plus récente)
data = db.iloc[::-1].reset_index(drop=True)

data = data[:-5] # sup

date_column = 'Date'
close_column = 'Price'

dates = pd.to_datetime(data[date_column])
close_prices = data[close_column]

# Décomposition en ondelettes
coeffs = pywt.wavedec(close_prices, 'db4', level=2)
cA2, cD2, cD1 = coeffs

approximation = pywt.upcoef('a', cA2, 'db4', level=2, take=len(close_prices))
detail2 = pywt.upcoef('d', cD2, 'db4', level=2, take=len(close_prices))
detail1 = pywt.upcoef('d', cD1, 'db4', level=1, take=len(close_prices))

import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


# Normalisation des données
scaler = MinMaxScaler(feature_range=(0, 1))
cD1_scaled = scaler.fit_transform(cD1.reshape(-1, 1))

# Création des fenêtres temporelles
def create_dataset(data, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i+window_size, 0])  # Séquence des données passées
        y.append(data[i+window_size, 0])  # Prochaine valeur (target)
    return np.array(X), np.array(y)

window_size = 10  # Utilisez les 10 dernières valeurs pour prédire la suivante
X, y = create_dataset(cD1_scaled, window_size)

# Reshape des données pour LSTM [échantillons, timesteps, features]
X = X.reshape(X.shape[0], X.shape[1], 1)

# Division en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

from keras.models import Sequential
from keras.layers import LSTM, Dense

# Définition du modèle LSTM
model = Sequential()
model.add(LSTM(units=50, return_sequences=False, input_shape=(X_train.shape[1], 1)))
model.add(Dense(1))  # Une seule sortie (la prochaine valeur de cD1)

# Compilation du modèle
model.compile(optimizer='adam', loss='mean_squared_error')

# Entraînement du modèle
model.fit(X_train, y_train, epochs=20, batch_size=32)

# Prédire la prochaine valeur de cD1 à partir de la dernière fenêtre de test
predicted_value_cD1 = model.predict(X_test[-1].reshape(1, window_size, 1))

# Inverse de la normalisation pour obtenir la valeur réelle
predicted_value_rescaled_cD1 = scaler.inverse_transform(predicted_value_cD1)

# Affichage de la valeur prédite
print(f"Prochaine valeur prédite pour cD1 : {predicted_value_rescaled_cD1[0][0]}")

# Dernière valeur de cA1 (approximation à un certain niveau)
last_cD1_value = cD1[-1]  # Dernière valeur de cA1

# Valeur prédite rééchelonnée (prédiction pour le prochain point)
predicted_value_rescaled_cD1 = predicted_value_rescaled_cD1[0][0]

# Comparaison entre la dernière valeur de cA1 et la valeur prédite
if predicted_value_rescaled_cD1 > last_cD1_value:
    print("Le signal devrait augmenter.")
else:
    print("Le signal devrait diminuer.")

# Normalisation des données pour cD2
scaler = MinMaxScaler(feature_range=(0, 1))
cD2_scaled = scaler.fit_transform(cD2.reshape(-1, 1))  # Remplacez cD2 par vos données de cD2

# Création des fenêtres temporelles pour cD2
def create_dataset(data, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i+window_size, 0])  # Séquence des données passées
        y.append(data[i+window_size, 0])  # Prochaine valeur (target)
    return np.array(X), np.array(y)

window_size = 10  # Utilisez les 10 dernières valeurs pour prédire la suivante
X, y = create_dataset(cD2_scaled, window_size)

# Reshape des données pour LSTM [échantillons, timesteps, features]
X = X.reshape(X.shape[0], X.shape[1], 1)

# Division en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Définition du modèle LSTM
model = Sequential()
model.add(LSTM(units=50, return_sequences=False, input_shape=(X_train.shape[1], 1)))
model.add(Dense(1))  # Une seule sortie (la prochaine valeur de cD2)

# Compilation du modèle
model.compile(optimizer='adam', loss='mean_squared_error')

# Entraînement du modèle
model.fit(X_train, y_train, epochs=20, batch_size=32)

# Prédire la prochaine valeur de cD2 à partir de la dernière fenêtre de test
predicted_value_cD2 = model.predict(X_test[-1].reshape(1, window_size, 1))

# Inverse de la normalisation pour obtenir la valeur réelle
predicted_value_rescaled_cD2 = scaler.inverse_transform(predicted_value_cD2)

# Affichage de la valeur prédite
print(f"Prochaine valeur prédite pour cD2 : {predicted_value_rescaled_cD2[0][0]}")

# Dernière valeur de cA1 (approximation à un certain niveau)
last_cD2_value = cD2[-1]  # Dernière valeur de cD2

# Valeur prédite rééchelonnée (prédiction pour le prochain point)
predicted_value_rescaled_cD2 = predicted_value_rescaled_cD2[0][0]

# Comparaison entre la dernière valeur de cA1 et la valeur prédite
if predicted_value_rescaled_cD2 > last_cD2_value:
    print("Le signal devrait augmenter.")
else:
    print("Le signal devrait diminuer.")

# Définir les valeurs à comparer
last_cD2_value = cD2[-1]  # Dernière valeur de cD2
last_cD1_value = cD1[-1]  # Dernière valeur de cD1
predicted_value_rescaled_cD1 = predicted_value_rescaled_cD1  # Prédiction pour cD1
predicted_value_rescaled_cD2 = predicted_value_rescaled_cD2  # Prédiction pour cD2

# Condition pour Achat, Vente ou Neutre
if last_cD2_value < predicted_value_rescaled_cD2 and last_cD1_value < predicted_value_rescaled_cD1:
    print("Achetez")
elif last_cD2_value >= predicted_value_rescaled_cD2 and last_cD1_value >= predicted_value_rescaled_cD2:
    print("N'achetez pas") # vente
else:
    print("N'achetez pas") # neutre

Epoch 1/20


/home/ab/Desktop/qiskit-env/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0987
Epoch 2/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0028
Epoch 3/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0014  
Epoch 4/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0012   
Epoch 5/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0015 
Epoch 6/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0019
Epoch 7/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0016  
Epoch 8/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0018 
Epoch 9/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0017
Epoch 10/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0015 
Epoch 11/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 9.4983e-04
Epoch 12/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0013   
Epoch 13/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0014 
Epoch 14/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0011
Epoch 15/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0017 
Epoch 16

/home/ab/Desktop/qiskit-env/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.1231
Epoch 2/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0092
Epoch 3/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0050
Epoch 4/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0025  
Epoch 5/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0032
Epoch 6/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0015  
Epoch 7/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0031
Epoch 8/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0026
Epoch 9/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0026
Epoch 10/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0024
Epoch 11/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0025  
Epoch 12/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0020    
Epoch 13/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0026
Epoch 14/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0025
Epoch 15/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0027    
Epoch 16/2

In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pywt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.lines import Line2D

file_path = '/home/ab/Desktop/MASTER DSEF/Dogecoin Historical Data_.ods'
db = pd.read_excel(file_path)

# Convertir la colonne 'Date' en format datetime en utilisant le format correct
db['Date'] = pd.to_datetime(db['Date'], format='%m/%d/%Y')

# Inverser l'ordre des données (de la plus ancienne à la plus récente)
data = db.iloc[::-1].reset_index(drop=True)

data = data[:-6] # sup

date_column = 'Date'
close_column = 'Price'

dates = pd.to_datetime(data[date_column])
close_prices = data[close_column]

# Décomposition en ondelettes
coeffs = pywt.wavedec(close_prices, 'db4', level=2)
cA2, cD2, cD1 = coeffs

approximation = pywt.upcoef('a', cA2, 'db4', level=2, take=len(close_prices))
detail2 = pywt.upcoef('d', cD2, 'db4', level=2, take=len(close_prices))
detail1 = pywt.upcoef('d', cD1, 'db4', level=1, take=len(close_prices))

import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


# Normalisation des données
scaler = MinMaxScaler(feature_range=(0, 1))
cD1_scaled = scaler.fit_transform(cD1.reshape(-1, 1))

# Création des fenêtres temporelles
def create_dataset(data, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i+window_size, 0])  # Séquence des données passées
        y.append(data[i+window_size, 0])  # Prochaine valeur (target)
    return np.array(X), np.array(y)

window_size = 10  # Utilisez les 10 dernières valeurs pour prédire la suivante
X, y = create_dataset(cD1_scaled, window_size)

# Reshape des données pour LSTM [échantillons, timesteps, features]
X = X.reshape(X.shape[0], X.shape[1], 1)

# Division en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

from keras.models import Sequential
from keras.layers import LSTM, Dense

# Définition du modèle LSTM
model = Sequential()
model.add(LSTM(units=50, return_sequences=False, input_shape=(X_train.shape[1], 1)))
model.add(Dense(1))  # Une seule sortie (la prochaine valeur de cD1)

# Compilation du modèle
model.compile(optimizer='adam', loss='mean_squared_error')

# Entraînement du modèle
model.fit(X_train, y_train, epochs=20, batch_size=32)

# Prédire la prochaine valeur de cD1 à partir de la dernière fenêtre de test
predicted_value_cD1 = model.predict(X_test[-1].reshape(1, window_size, 1))

# Inverse de la normalisation pour obtenir la valeur réelle
predicted_value_rescaled_cD1 = scaler.inverse_transform(predicted_value_cD1)

# Affichage de la valeur prédite
print(f"Prochaine valeur prédite pour cD1 : {predicted_value_rescaled_cD1[0][0]}")

# Dernière valeur de cA1 (approximation à un certain niveau)
last_cD1_value = cD1[-1]  # Dernière valeur de cA1

# Valeur prédite rééchelonnée (prédiction pour le prochain point)
predicted_value_rescaled_cD1 = predicted_value_rescaled_cD1[0][0]

# Comparaison entre la dernière valeur de cA1 et la valeur prédite
if predicted_value_rescaled_cD1 > last_cD1_value:
    print("Le signal devrait augmenter.")
else:
    print("Le signal devrait diminuer.")

# Normalisation des données pour cD2
scaler = MinMaxScaler(feature_range=(0, 1))
cD2_scaled = scaler.fit_transform(cD2.reshape(-1, 1))  # Remplacez cD2 par vos données de cD2

# Création des fenêtres temporelles pour cD2
def create_dataset(data, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i+window_size, 0])  # Séquence des données passées
        y.append(data[i+window_size, 0])  # Prochaine valeur (target)
    return np.array(X), np.array(y)

window_size = 10  # Utilisez les 10 dernières valeurs pour prédire la suivante
X, y = create_dataset(cD2_scaled, window_size)

# Reshape des données pour LSTM [échantillons, timesteps, features]
X = X.reshape(X.shape[0], X.shape[1], 1)

# Division en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Définition du modèle LSTM
model = Sequential()
model.add(LSTM(units=50, return_sequences=False, input_shape=(X_train.shape[1], 1)))
model.add(Dense(1))  # Une seule sortie (la prochaine valeur de cD2)

# Compilation du modèle
model.compile(optimizer='adam', loss='mean_squared_error')

# Entraînement du modèle
model.fit(X_train, y_train, epochs=20, batch_size=32)

# Prédire la prochaine valeur de cD2 à partir de la dernière fenêtre de test
predicted_value_cD2 = model.predict(X_test[-1].reshape(1, window_size, 1))

# Inverse de la normalisation pour obtenir la valeur réelle
predicted_value_rescaled_cD2 = scaler.inverse_transform(predicted_value_cD2)

# Affichage de la valeur prédite
print(f"Prochaine valeur prédite pour cD2 : {predicted_value_rescaled_cD2[0][0]}")

# Dernière valeur de cA1 (approximation à un certain niveau)
last_cD2_value = cD2[-1]  # Dernière valeur de cD2

# Valeur prédite rééchelonnée (prédiction pour le prochain point)
predicted_value_rescaled_cD2 = predicted_value_rescaled_cD2[0][0]

# Comparaison entre la dernière valeur de cA1 et la valeur prédite
if predicted_value_rescaled_cD2 > last_cD2_value:
    print("Le signal devrait augmenter.")
else:
    print("Le signal devrait diminuer.")

# Définir les valeurs à comparer
last_cD2_value = cD2[-1]  # Dernière valeur de cD2
last_cD1_value = cD1[-1]  # Dernière valeur de cD1
predicted_value_rescaled_cD1 = predicted_value_rescaled_cD1  # Prédiction pour cD1
predicted_value_rescaled_cD2 = predicted_value_rescaled_cD2  # Prédiction pour cD2

# Condition pour Achat, Vente ou Neutre
if last_cD2_value < predicted_value_rescaled_cD2 and last_cD1_value < predicted_value_rescaled_cD1:
    print("Achetez")
elif last_cD2_value >= predicted_value_rescaled_cD2 and last_cD1_value >= predicted_value_rescaled_cD2:
    print("N'achetez pas") # vente
else:
    print("N'achetez pas") # neutre

Epoch 1/20


/home/ab/Desktop/qiskit-env/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0641
Epoch 2/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0023
Epoch 3/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0016
Epoch 4/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0012   
Epoch 5/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0019 
Epoch 6/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0015
Epoch 7/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0017
Epoch 8/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0017   
Epoch 9/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0019  
Epoch 10/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0022
Epoch 11/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0015  
Epoch 12/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0012   
Epoch 13/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 9.6707e-04
Epoch 14/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0016  
Epoch 15/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0015
Epoch 16/2

/home/ab/Desktop/qiskit-env/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.1823
Epoch 2/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0117
Epoch 3/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0041
Epoch 4/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0025
Epoch 5/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0023   
Epoch 6/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0027   
Epoch 7/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0019   
Epoch 8/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0026
Epoch 9/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0016   
Epoch 10/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0022   
Epoch 11/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0025    
Epoch 12/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0031
Epoch 13/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0024   
Epoch 14/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0029   
Epoch 15/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0017
Epoch

In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pywt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.lines import Line2D

file_path = '/home/ab/Desktop/MASTER DSEF/Dogecoin Historical Data_.ods'
db = pd.read_excel(file_path)

# Convertir la colonne 'Date' en format datetime en utilisant le format correct
db['Date'] = pd.to_datetime(db['Date'], format='%m/%d/%Y')

# Inverser l'ordre des données (de la plus ancienne à la plus récente)
data = db.iloc[::-1].reset_index(drop=True)

data = data[:-7] # sup

date_column = 'Date'
close_column = 'Price'

dates = pd.to_datetime(data[date_column])
close_prices = data[close_column]

# Décomposition en ondelettes
coeffs = pywt.wavedec(close_prices, 'db4', level=2)
cA2, cD2, cD1 = coeffs

approximation = pywt.upcoef('a', cA2, 'db4', level=2, take=len(close_prices))
detail2 = pywt.upcoef('d', cD2, 'db4', level=2, take=len(close_prices))
detail1 = pywt.upcoef('d', cD1, 'db4', level=1, take=len(close_prices))

import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


# Normalisation des données
scaler = MinMaxScaler(feature_range=(0, 1))
cD1_scaled = scaler.fit_transform(cD1.reshape(-1, 1))

# Création des fenêtres temporelles
def create_dataset(data, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i+window_size, 0])  # Séquence des données passées
        y.append(data[i+window_size, 0])  # Prochaine valeur (target)
    return np.array(X), np.array(y)

window_size = 10  # Utilisez les 10 dernières valeurs pour prédire la suivante
X, y = create_dataset(cD1_scaled, window_size)

# Reshape des données pour LSTM [échantillons, timesteps, features]
X = X.reshape(X.shape[0], X.shape[1], 1)

# Division en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

from keras.models import Sequential
from keras.layers import LSTM, Dense

# Définition du modèle LSTM
model = Sequential()
model.add(LSTM(units=50, return_sequences=False, input_shape=(X_train.shape[1], 1)))
model.add(Dense(1))  # Une seule sortie (la prochaine valeur de cD1)

# Compilation du modèle
model.compile(optimizer='adam', loss='mean_squared_error')

# Entraînement du modèle
model.fit(X_train, y_train, epochs=20, batch_size=32)

# Prédire la prochaine valeur de cD1 à partir de la dernière fenêtre de test
predicted_value_cD1 = model.predict(X_test[-1].reshape(1, window_size, 1))

# Inverse de la normalisation pour obtenir la valeur réelle
predicted_value_rescaled_cD1 = scaler.inverse_transform(predicted_value_cD1)

# Affichage de la valeur prédite
print(f"Prochaine valeur prédite pour cD1 : {predicted_value_rescaled_cD1[0][0]}")

# Dernière valeur de cA1 (approximation à un certain niveau)
last_cD1_value = cD1[-1]  # Dernière valeur de cA1

# Valeur prédite rééchelonnée (prédiction pour le prochain point)
predicted_value_rescaled_cD1 = predicted_value_rescaled_cD1[0][0]

# Comparaison entre la dernière valeur de cA1 et la valeur prédite
if predicted_value_rescaled_cD1 > last_cD1_value:
    print("Le signal devrait augmenter.")
else:
    print("Le signal devrait diminuer.")

# Normalisation des données pour cD2
scaler = MinMaxScaler(feature_range=(0, 1))
cD2_scaled = scaler.fit_transform(cD2.reshape(-1, 1))  # Remplacez cD2 par vos données de cD2

# Création des fenêtres temporelles pour cD2
def create_dataset(data, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i+window_size, 0])  # Séquence des données passées
        y.append(data[i+window_size, 0])  # Prochaine valeur (target)
    return np.array(X), np.array(y)

window_size = 10  # Utilisez les 10 dernières valeurs pour prédire la suivante
X, y = create_dataset(cD2_scaled, window_size)

# Reshape des données pour LSTM [échantillons, timesteps, features]
X = X.reshape(X.shape[0], X.shape[1], 1)

# Division en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Définition du modèle LSTM
model = Sequential()
model.add(LSTM(units=50, return_sequences=False, input_shape=(X_train.shape[1], 1)))
model.add(Dense(1))  # Une seule sortie (la prochaine valeur de cD2)

# Compilation du modèle
model.compile(optimizer='adam', loss='mean_squared_error')

# Entraînement du modèle
model.fit(X_train, y_train, epochs=20, batch_size=32)

# Prédire la prochaine valeur de cD2 à partir de la dernière fenêtre de test
predicted_value_cD2 = model.predict(X_test[-1].reshape(1, window_size, 1))

# Inverse de la normalisation pour obtenir la valeur réelle
predicted_value_rescaled_cD2 = scaler.inverse_transform(predicted_value_cD2)

# Affichage de la valeur prédite
print(f"Prochaine valeur prédite pour cD2 : {predicted_value_rescaled_cD2[0][0]}")

# Dernière valeur de cA1 (approximation à un certain niveau)
last_cD2_value = cD2[-1]  # Dernière valeur de cD2

# Valeur prédite rééchelonnée (prédiction pour le prochain point)
predicted_value_rescaled_cD2 = predicted_value_rescaled_cD2[0][0]

# Comparaison entre la dernière valeur de cA1 et la valeur prédite
if predicted_value_rescaled_cD2 > last_cD2_value:
    print("Le signal devrait augmenter.")
else:
    print("Le signal devrait diminuer.")

# Définir les valeurs à comparer
last_cD2_value = cD2[-1]  # Dernière valeur de cD2
last_cD1_value = cD1[-1]  # Dernière valeur de cD1
predicted_value_rescaled_cD1 = predicted_value_rescaled_cD1  # Prédiction pour cD1
predicted_value_rescaled_cD2 = predicted_value_rescaled_cD2  # Prédiction pour cD2

# Condition pour Achat, Vente ou Neutre
if last_cD2_value < predicted_value_rescaled_cD2 and last_cD1_value < predicted_value_rescaled_cD1:
    print("Achetez")
elif last_cD2_value >= predicted_value_rescaled_cD2 and last_cD1_value >= predicted_value_rescaled_cD2:
    print("N'achetez pas") # vente
else:
    print("N'achetez pas") # neutre

Epoch 1/20


/home/ab/Desktop/qiskit-env/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.0775
Epoch 2/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0032
Epoch 3/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0017  
Epoch 4/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0020  
Epoch 5/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0028
Epoch 6/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0016
Epoch 7/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0014  
Epoch 8/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0018  
Epoch 9/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0017  
Epoch 10/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0014  
Epoch 11/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0014  
Epoch 12/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0015  
Epoch 13/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0018
Epoch 14/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0015  
Epoch 15/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0034
Epoch 16/20


/home/ab/Desktop/qiskit-env/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.2017
Epoch 2/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0074
Epoch 3/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0031    
Epoch 4/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0029
Epoch 5/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0018    
Epoch 6/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0021
Epoch 7/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0019   
Epoch 8/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0040
Epoch 9/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0028   
Epoch 10/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0048
Epoch 11/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0026
Epoch 12/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0021   
Epoch 13/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0020
Epoch 14/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0023    
Epoch 15/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0019
Epoch 16